In [1]:
import os

In [2]:

%pwd

'd:\\Personal\\Projects_Computer_Science_And_Engineering\\RaunakMLProjects\\End-to-end-ML-Project\\research'

In [ ]:
os.chdir("../")

In [7]:
%cd research

d:\Personal\Projects_Computer_Science_And_Engineering\RaunakMLProjects\End-to-end-ML-Project\research


d:\Personal\Projects_Computer_Science_And_Engineering\RaunakMLProjects\venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [77]:
%pwd

'd:\\Personal\\Projects_Computer_Science_And_Engineering\\RaunakMLProjects\\End-to-end-ML-Project'

In [98]:
## working with entity:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    # Define the fields with type annotations
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [99]:
# updating configuration manger configuration .py
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml,create_directories



In [100]:
#config manager class
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config



In [101]:
#dta inejction componenet
import os
import urllib.request as request
import zipfile
from src.mlProject import logger
from src.mlProject.utils.common import get_size



In [102]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config= config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [104]:
#update pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-30 12:15:15,732: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-30 12:15:15,735: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-30 12:15:15,737: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-30 12:15:15,739: INFO: common: created directory at: artifacts]
[2024-12-30 12:15:15,740: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-30 12:15:15,741: INFO: 2590978666: File already exists of size: ~ 2400 KB]


In [105]:
#before moving further need to ocnvert this notebook to modular coding 